In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.models as models
import numpy as np

In [2]:
class Net(nn.Module):
    #define the layers
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(16*53*53, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        self.relu = nn.ReLU()
        
    #concatenate these layers
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 16*53*53)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [3]:
# coding:utf8
from torch import nn


class NetG(nn.Module):
    """
    生成器定义
    """

    def __init__(self, opt):
        super(NetG, self).__init__()
        ngf = opt.ngf  # 生成器feature map数

        self.main = nn.Sequential(
            # 输入是一个nz维度的噪声，我们可以认为它是一个1*1*nz的feature map
            nn.ConvTranspose2d(opt.nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # 上一步的输出形状：(ngf*8) x 4 x 4

            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # 上一步的输出形状： (ngf*4) x 8 x 8

            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # 上一步的输出形状： (ngf*2) x 16 x 16

            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # 上一步的输出形状：(ngf) x 32 x 32

            nn.ConvTranspose2d(ngf, 3, 5, 3, 1, bias=False),
            nn.Tanh()  # 输出范围 -1~1 故而采用Tanh
            # 输出形状：3 x 96 x 96
        )

    def forward(self, input):
        return self.main(input)


class NetD(nn.Module):
    """
    判别器定义
    """

    def __init__(self, opt):
        super(NetD, self).__init__()
        ndf = opt.ndf
        self.main = nn.Sequential(
            # 输入 3 x 96 x 96
            nn.Conv2d(3, ndf, 5, 3, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # 输出 (ndf) x 32 x 32

            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # 输出 (ndf*2) x 16 x 16

            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # 输出 (ndf*4) x 8 x 8

            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # 输出 (ndf*8) x 4 x 4

            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()  # 输出一个数(概率)
        )

    def forward(self, input):
        return self.main(input).view(-1)
    
class Config(object):
    data_path = 'data/'  # 数据集存放路径
    num_workers = 4  # 多进程加载数据所用的进程数
    image_size = 96  # 图片尺寸
    batch_size = 256
    max_epoch = 200
    lr1 = 2e-4  # 生成器的学习率
    lr2 = 2e-4  # 判别器的学习率
    beta1 = 0.5  # Adam优化器的beta1参数
    gpu = True  # 是否使用GPU
    nz = 100  # 噪声维度
    ngf = 64  # 生成器feature map数
    ndf = 64  # 判别器feature map数

    save_path = 'imgs/'  # 生成图片保存路径

    vis = True  # 是否使用visdom可视化
    env = 'GAN'  # visdom的env
    plot_every = 20  # 每间隔20 batch，visdom画图一次

    debug_file = '/tmp/debuggan'  # 存在该文件则进入debug模式
    d_every = 1  # 每1个batch训练一次判别器
    g_every = 5  # 每5个batch训练一次生成器
    save_every = 10  # 没10个epoch保存一次模型
    netd_path = None  # 'checkpoints/netd_.pth' #预训练模型
    netg_path = None  # 'checkpoints/netg_211.pth'

    # 只测试不训练
    gen_img = 'result.png'
    # 从512张生成的图片中保存最好的64张
    gen_num = 64
    gen_search_num = 512
    gen_mean = 0  # 噪声的均值
    gen_std = 1  # 噪声的方差


opt = Config()

In [4]:
def my_hook_function(self, input, output):
    op_name = str(self.__class__.__name__)
    input_size = list(input[0].size())
    output_size = list(output.size())
    params = 0
    global layers_output
    global all_params
    global all_MACs
    
    if op_name == 'Conv2d':
        batch_size = input_size[0]
        kernel_size = list(self.kernel_size)
        padding = self.padding[0]
        stride = self.stride[0]
        groups = self.groups
        
        macs = batch_size *  kernel_size[0] * kernel_size[1] * input_size[1] * (output_size[1]//groups) * output_size[2] * output_size[3]
        # kernel_size * kernel_size * input_channels * output_channels * output_image_width * output_image_length
        
        active_elements_count = batch_size * output_size[2] * output_size[3]
        params = 1
        for param in self.parameters():
            for i in list(param.size()):
                params *= i
            break

        
        layers_output.append([op_name, input_size, output_size, params, macs])
        all_params.append(params)
        all_MACs.append(macs)
        
    elif op_name == 'Linear':
        in_features = self.in_features
        out_features = self.out_features
        
        macs = in_features * out_features
        params = 1
        for param in self.parameters():
            for i in list(param.size()):
                params *= i
            break
            
        layers_output.append([op_name, input_size, output_size, params, macs])
        all_params.append(params)
        all_MACs.append(macs)

    elif op_name == 'MaxPool2d' or op_name == 'AdaptiveAvgPool2d':
        macs = int(np.prod(np.array(input_size)))
        layers_output.append([op_name, input_size, output_size, 0, macs])
        all_params.append(0)
        all_MACs.append(macs)

    elif op_name == 'BatchNorm2d':
        self.affine
        batch_flops = np.prod(np.array(input_size))
        
        macs = np.prod(np.array(input_size))
        if self.affine:
            batch_flops *= 2
        params = 1 
        for param in self.parameters():
            for i in list(param.size()):
                params *= i
            break
        layers_output.append([op_name, input_size, output_size, params, batch_flops])
        all_params.append(params)
        all_MACs.append(batch_flops)

    elif op_name == 'ReLU' or op_name == 'ReLU6' or op_name == 'LeakyReLU':
        active_elements_count = output.numel()
        macs = int(active_elements_count)
        layers_output.append([op_name, input_size, output_size, 0, macs])
        all_params.append(0)
        all_MACs.append(macs)
    
    elif op_name == 'ConvTranspose2d':
        batch_size = input_size[0]
        kernel_size = list(self.kernel_size)
        padding = self.padding[0]
        stride = self.stride[0]
        groups = self.groups
        
        macs = kernel_size[0] * kernel_size[1] * input_size[1] * (output_size[1]//groups) * batch_size * input_size[2] * input_size[3]
        # kernel_size * kernel_size * input_channels * output_channels * input_image_width * input_image_length
        
        bias_flops = 0
        if self.bias is not None:
            output_height, output_width = output.shape[2:]
            bias_flops = out_channels * batch_size * output_height * output_height
        params = 1
        for param in self.parameters():
            for i in list(param.size()):
                params *= i
            break

        
        layers_output.append([op_name, input_size, output_size, params, macs + bias_flops])
        all_params.append(params)
        all_MACs.append(macs + bias_flops)
        
    elif op_name == 'Tanh':
        layers_output.append([op_name, input_size, output_size, 0, 0])
        all_params.append(0)
        all_MACs.append(0)
        
        
    else:
        layers_output.append([op_name, input_size, output_size, None, None])
        all_params.append(0)
        all_MACs.append(0)
    

In [5]:
#model = models.vgg16_bn()
model = models.resnet18()
#model = models.mobilenet_v2()
#model = NetG(opt)


def to_register(layers):
    if list(layers.children()) == []:
        layers.register_forward_hook(my_hook_function)
    else:
        for i in list(layers.children()):
            to_register(i)

to_register(model)

In [6]:
input_data = torch.randn(1, 3, 224, 224)
#input_data = torch.randn(1, 100, 1, 1)

In [7]:
layers_output = []
all_params = []
all_MACs = []

print('Lab 1-3:')
print('%-25s%-25s%-25s%-25s%-15s' % ('op_type', 'input_shape', 'output_shape', 'params', 'MACs'))
print('--------------------------------------------------------------------------------------------------------------------')
output = model(input_data)
for i in layers_output:
    print('%-25s%-25s%-25s%-25s%-15s' % (i[0], i[1], i[2], i[3], i[4]))
print('--------------------------------------------------------------------------------------------------------------------')
print('Total params: %.3f M' % (sum(all_params) / 1000000))
print('Total MACs: %.3f G' % (sum(all_MACs) / 1000000000))
print('Total FLOPs: %.3f G' % (sum(all_MACs)*2 / 1000000000))

Lab 1-3:
op_type                  input_shape              output_shape             params                   MACs           
--------------------------------------------------------------------------------------------------------------------
Conv2d                   [1, 3, 224, 224]         [1, 64, 112, 112]        9408                     118013952      
BatchNorm2d              [1, 64, 112, 112]        [1, 64, 112, 112]        64                       1605632        
ReLU                     [1, 64, 112, 112]        [1, 64, 112, 112]        0                        802816         
MaxPool2d                [1, 64, 112, 112]        [1, 64, 56, 56]          0                        802816         
Conv2d                   [1, 64, 56, 56]          [1, 64, 56, 56]          36864                    115605504      
BatchNorm2d              [1, 64, 56, 56]          [1, 64, 56, 56]          64                       401408         
ReLU                     [1, 64, 56, 56]          [1, 64, 56, 